xgboostをインストール済みのクラスターで実行

In [2]:
import pandas as pd 
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

dbutils.fs.mount(
  source = "wasbs://housedata@artificialdatastorage.blob.core.windows.net",
  mount_point = "/mnt/housedata",
  extra_configs = {"fs.azure.account.key.artificialdatastorage.blob.core.windows.net":'/enKgvgod8Mk5cdhFlWfGrc1A/Ho73kn5LSjPUdMxS4Aj731FW0Fkmrw1qqa+Lay2uCkbVGkBO0vK3NZv6GB6g=='})


--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3388580873466491> in <module> () 
 8 source = "wasbs://housedata@artificialdatastorage.blob.core.windows.net" , 
 9 mount_point = "/mnt/housedata" , 
 ---> 10 extra_configs = {"fs.azure.account.key.artificialdatastorage.blob.core.windows.net":'/enKgvgod8Mk5cdhFlWfGrc1A/Ho73kn5LSjPUdMxS4Aj731FW0Fkmrw1qqa+Lay2uCkbVGkBO0vK3NZv6GB6g=='})
 
 /local_disk0/tmp/1550194763510-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 302 exc . __context__ = None 
 303 exc . __cause__ = None 
 --> 304 raise exc
 305 return f_with_exception_handling
 306 

 ExecutionError : An error occurred while calling o190.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/housedata; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/housedata
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:54)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:454)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/housedata
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:127)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:66)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:83)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:82)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:82)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:272)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:252)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:42)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:58)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:58)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:38)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$2(UsageLogging.scala:359)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:227)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:13)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:268)
	at com.databricks.logging.UsageLogging.withA

In [3]:
housedata_df = spark.read.format("csv").option("header", "true").load("/mnt/housedata/housedata.csv")
housedata = housedata_df.toPandas()

In [4]:
#display(housedata_df)
housedata.head(5)

Out[10]: 
 price bedrooms ... sqft_living15 sqft_lot15
0 221900.0 3 ... 1340 5650
1 538000.0 3 ... 1690 7639
2 180000.0 2 ... 2720 8062
3 604000.0 4 ... 1360 5000
4 510000.0 3 ... 1800 7503

[5 rows x 19 columns]

In [5]:
#housedata = housedata.drop("_c0",axis=1)
housedata.head(3)
X, y = housedata.iloc[:,1:],housedata.iloc[:,0]
X = X.astype("float64")
y = y.astype("float64")
X.head(3)

Out[11]: 
 bedrooms bathrooms ... sqft_living15 sqft_lot15
0 3.0 1.00 ... 1340.0 5650.0
1 3.0 2.25 ... 1690.0 7639.0
2 2.0 1.00 ... 2720.0 8062.0

[3 rows x 18 columns]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [7]:
alphas = []
for alpha in range(0,10000000,100000):
  alphas.append(alpha/100000)

tasks = []
for alpha in alphas:
    for est in range(10,50,1):
      tasks.append((alpha, est))

In [8]:
tasksRDD = sc.parallelize(tasks, numSlices = len(tasks))

In [9]:
X_train_Broadcast = sc.broadcast(X_train)
y_train_Broadcast = sc.broadcast(y_train)

X_test_Broadcast = sc.broadcast(X_test)
y_test_Broadcast = sc.broadcast(y_test)

In [10]:
def trainXGBModel(a, n_est):
  
  X_train_local = X_train_Broadcast.value
  y_train_local = y_train_Broadcast.value
  
  X_test_local = X_test_Broadcast.value
  y_test_local = y_test_Broadcast.value
   
  # Train the model, and score it
  xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 30, alpha = a, n_estimators = n_est)
  xg_reg.fit(X_train_local, y_train_local)
  score = xg_reg.score(X_test_local, y_test_local)
  return xg_reg, score, alpha, n_est

In [11]:
trainedModelAndScores = tasksRDD.map(lambda alpha_est: trainXGBModel(alpha_est[0], alpha_est[1]))
trainedModelAndScores.cache()
trainedModelAndScores.count()

Out[17]: 4000

In [12]:
X_train_Broadcast.unpersist()
y_train_Broadcast.unpersist()
X_test_Broadcast.unpersist()
y_test_Broadcast.unpersist()

In [13]:
# Collect the results.
allScores = trainedModelAndScores.map(lambda x: (x[1], x[2], x[3])).collect()

# find best parameter
bestScore = -1
bestAlpha = -1
bestN_est = -1

for score, alpha, n_est in allScores:
  if score > bestScore:
    bestScore = score
    bestAlpha = alpha
    bestN_est = n_est

In [14]:
print("best_score:"+str(bestScore))
print("random:"+str(allScores[int(np.random.rand()*len(allScores))][0]))

best_score:0.8747621736173873
RandomParameter:0.8416121241201708